# Import and process all corresponding data from the key paper. 

#### * Key Paper: https://www.sciencedirect.com/science/article/abs/pii/S1544612315001038

    * Title: Bitcoin, gold and the dollar – A GARCH volatility analysis

    * Author: Anne Haubo Dyhrberg

    * Publication: Finance Research Letters

    * Publisher: Elsevier

    * Date: February 2016

Will be importing 

1. FEDFUND.csv - monthly effective fed rate. (data source: FRED)

2. GOLDPMGBD228NLBM.csv - daily Gold Fixing Price 3:00 PM (London Time) in London Bullion Market, based in USD. (data source: FRED)

3. KOSPI_index_20060512-20190612.xls - daily KOSPI index. (data source: KRX)

4. BOK_8.8.1.1원달러환율.csv - daily KRW/USD Exchange Rate (data source: BOK)

5. BTC.pkl - scraped daily BTC/USD data (includes volume & market cap) (data source: coinmarketcap.com)

and process them to daily dataframe from 2014.01.01 ~ 

In [1]:
import pandas as pd

1. Fed fund rate data

In [37]:
fed = pd.read_csv("./keypaperdata/FEDFUNDS.csv")
fed.head()

,DATE,FEDFUNDS
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.06
3,1954-10-01,0.85
4,1954-11-01,0.83


In [38]:
fed.columns = ['date', 'fed_rate']
fed.date = pd.to_datetime(fed.date)
fed.set_index('date', inplace=True)

fed = fed[fed.index >= pd.to_datetime('2014-01-01')]
fed.head()

,fed_rate
date,
2014-01-01,0.07
2014-02-01,0.07
2014-03-01,0.08
2014-04-01,0.09
2014-05-01,0.09


In [85]:
fed.dtypes

fed_rate    float64
dtype: object

2. Gold

In [69]:
gold = pd.read_csv('./keypaperdata/GOLDPMGBD228NLBM.csv')
gold.head()

,DATE,GOLDPMGBD228NLBM
0,2009-06-11,947.500
1,2009-06-12,937.250
2,2009-06-15,932.250
3,2009-06-16,934.000
4,2009-06-17,930.500


In [70]:
gold.columns = ['date', 'gold_usd']
gold.date = pd.to_datetime(gold.date)
gold.set_index('date', inplace=True)

gold.gold_usd.replace(to_replace='.', method='pad', inplace=True)

gold = gold[gold.index >= pd.to_datetime('2014-01-01')]
gold.gold_usd = pd.to_numeric(gold.gold_usd)
gold.head()

,gold_usd
date,
2014-01-01,1204.50
2014-01-02,1225.00
2014-01-03,1234.50
2014-01-06,1246.25
2014-01-07,1227.50


In [84]:
gold.dtypes

gold_usd    float64
dtype: object

3. KOSPI

In [81]:
kospi = pd.read_excel(pd.ExcelFile("./keypaperdata/KOSPI_index_20060512-20190612.xls"), 'Sheet1', thousands=',')
kospi.head()

,일자,현재지수,대비,등락률(%),배당수익률(%),주가이익비율,주가자산비율,시가지수,고가지수,저가지수,거래량(천주),거래대금(백만원),상장시가총액(백만원)
0,2006/05/12,1445.20,-19.50,-1.33,1.58,11.91,1.49,1448.57,1454.14,1429.92,260776,3755880,705654732
1,2006/05/15,1413.98,-31.22,-2.16,1.61,11.65,1.46,1424.58,1424.58,1406.88,250685,3936334,690651191
2,2006/05/16,1382.11,-31.87,-2.25,1.65,11.39,1.42,1415.81,1416.45,1373.52,285855,4439085,675119442
3,2006/05/17,1401.47,19.36,1.40,1.63,11.55,1.44,1386.93,1403.68,1386.93,270248,3829181,684397074
4,2006/05/18,1365.15,-36.32,-2.59,1.67,11.24,1.40,1360.76,1366.78,1356.04,355783,3911213,666633613


In [82]:
kospi = kospi[['일자', '현재지수']]
kospi.columns = ['date', 'kospi']
kospi.date = pd.to_datetime(kospi.date)
kospi.set_index('date', inplace=True)

kospi = kospi[kospi.index >= pd.to_datetime('2014-01-01')]
kospi.head()

,kospi
date,
2014-01-02,1967.19
2014-01-03,1946.14
2014-01-06,1953.28
2014-01-07,1959.44
2014-01-08,1958.96


In [83]:
kospi.dtypes

kospi    float64
dtype: object

4. KRW/USD exchange rate

In [86]:
# OSError raised if engine='python' is not used. 
krwusd = pd.read_csv('./keypaperdata/BOK_8.8.1.1원달러환율.csv', skiprows=3, engine='python', thousands=',')
krwusd.head()

,변환,원자료
0,2011/05/01,NaN
1,2011/05/02,1073.3
2,2011/05/03,1066.8
3,2011/05/04,1070.2
4,2011/05/05,NaN


In [87]:
krwusd.columns = ['date', 'ex_rate']
krwusd.date = pd.to_datetime(krwusd.date)
krwusd.set_index('date', inplace=True)

* pd.Dataframe.fillna(method=)

    - pad / ffill: propagate last valid observation forward to next valid
    - backfill / bfill: use NEXT valid observation to fill gap

In [88]:
# pd.DataFrame.fillna?
krwusd.ex_rate.fillna(method='ffill', inplace=True)

krwusd = krwusd[krwusd.index >= pd.to_datetime('2014-01-01')]
krwusd.head()

,ex_rate
date,
2014-01-01,1055.3
2014-01-02,1055.3
2014-01-03,1050.4
2014-01-04,1050.4
2014-01-05,1050.4


In [89]:
krwusd.dtypes

ex_rate    float64
dtype: object

5. BTC

In [51]:
btc = pd.read_pickle('./keypaperdata/BTC.pkl')
btc.head()

,Date,Open,High,Low,Close,Volume_USD,MarketCap_USD,Volume_BTC,MarketCap_BTC
0,2019. 06. 12.,7925.43,8196.65,7862.36,8145.86,19034432883,144634684711,2.336700e+06,1.775561e+07
1,2019. 06. 11.,8004.24,8026.39,7772.80,7927.71,17107279932,140748537530,2.157909e+06,1.775400e+07
2,2019. 06. 10.,7692.28,8031.91,7586.73,8000.33,18689275117,142023351079,2.336063e+06,1.775219e+07
3,2019. 06. 09.,7949.67,7975.97,7583.22,7688.08,16610726547,136465098533,2.160582e+06,1.775022e+07
4,2019. 06. 08.,8036.77,8076.89,7837.61,7954.13,16522722810,141172841635,2.077251e+06,1.774837e+07


In [52]:
btc = btc[['Date', 'Close', 'Volume_USD', 'MarketCap_USD', 'Volume_BTC', 'MarketCap_BTC']]
btc.columns = ['date', 'close', 'vol_usd', 'mktcap_usd', 'vol_usd', 'mktcap_usd']
btc.date = pd.to_datetime(btc.date)
btc.set_index('date', inplace=True)
btc = btc.iloc[::-1]
btc.head()

,close,vol_usd,mktcap_usd,vol_usd,mktcap_usd
date,,,,,
2014-01-01,771.40,22489400,9403308145,29154.005704,1.218992e+07
2014-01-02,802.39,38489500,9781073921,47968.568900,1.218993e+07
2014-01-03,818.72,37810100,9980135396,46181.966973,1.218992e+07
2014-01-04,859.51,38005000,10477362437,44217.053903,1.218993e+07
2014-01-05,933.53,72898496,11379660685,78089.076945,1.218992e+07


In [90]:
btc.dtypes

close         float64
vol_usd         int64
mktcap_usd      int64
vol_usd       float64
mktcap_usd    float64
dtype: object